In [1]:
%reload_ext autoreload
%autoreload 2

from qwanta import Tuner, QuantumNetwork, Configuration, Xperiment
from qwanta import QuantumNetwork, Configuration, Xperiment, Tuner
import pandas as pd
import ast
import numpy as np
import networkx as nx
from qwanta.Qubit import PhysicalQubit
import seaborn as sns
import matplotlib.pyplot as plt

### Define configuration and read the network timeline

As of current implementation, the timeline should be format as in csv or excel file to let Xperiment to be able to read it.
However, Xperiment is just a class wrapping that does the validation of the timeline and the network, so you can use any other format as long as you can read it and pass it to the Xperiment class.

In [22]:
# Hyperparameters

loss = 0.1 # dB/km
depo_prob = 0
gate_error = 0.1 
measurement_error = 0
memory_time = np.inf # second

num_hops = 2
num_nodes = num_hops + 1

node_info = {f'Node {i}': {'coordinate': (int(i*100), 0, 0)} for i in range(num_nodes)}
edge_info = {
    (f'Node {i}', f'Node {i+1}'): {
    'connection-type': 'Space',
    'depolarlizing error': [1 - depo_prob, depo_prob/3, depo_prob/3, depo_prob/3],
    'loss': loss,
    'light speed': 300000,
    'Pulse rate': 0.00001,
    f'Node {i}':{
        'gate error': gate_error,
        'measurement error': measurement_error,
        'memory function': memory_time
    },
    f'Node {i+1}':{
        'gate error': gate_error,
        'measurement error': measurement_error,
        'memory function': memory_time
    },
    }
for i in range(num_hops)}

exps = Xperiment(
    timelines_path = 'network/exper_id3_selectedStats_2hops.xlsx',
    nodes_info_exp = node_info,
    edges_info_exp = edge_info,
    gate_error = gate_error,
    measurement_error = measurement_error,
    memory_time = memory_time,
    # strategies_list=['0G']
)

### To validate the experiment configuration, and network topology

You can validate the experiment configuration and network topology by running the following command. 
If vis is set to True, the network topology will be visualized using pyvis library, you could see the network topology in the browser using the generated html file.
If get_table is set to True, the network topology will be printed in table format show the summary of validation.
If show_message is set to True, the validation message will be printed.

In [16]:
exps.validate(get_table=True, vis=True)

,Experiment,Resource-reachable,loss,depolarizing error,gate error,memory error,measurement error,Network
0,0G,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
1,1G-Ss-Dp,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
2,2G-NonLocalCNOT,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
3,1-2G-DirectedEncoded,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
4,HG-END2ENDPurifiedEncoded,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
5,1-2G-DirectedEncoded-Perfect,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
6,2G-NonLocalCNOT-Perfect,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED


### To run the experiment
You could run the experiment by running the following command.
Set save_result to True to save the experiment result to the experiment result folder.

In [23]:
results = exps.execute()

### Let us see the result of 0G experiment
The result dictionary contains the following keys:

In [25]:
for key in results['0G'].keys():
    print(key)

numPhysicalBuffer
numInternalEncodingBuffer
numInternalDetectingBuffer
numInternalInterfaceBuffer
memory_function
gate_error
measurement_error
timeline
experiment
light_speed_in_fiber
message
g
result_path
label_recorded
collectFidelityHistory
simulation_time
coor_system
nodes_info
NetworkTopology
throughtputEdges
fidelity
Resources Produced
Base Resources Attempt
Resource Used in Fidelity Estimation
Time used
Fidelity Estimation Time
Fidelity History
Qubits waiting time
Commutation inforamtion
Simulation log


In [43]:
def visualize(result):
    # to_print = ['fidelity', 'simulation_time', 'Resources Produced', 'Base Resources Attempt', 'Resource Used in Fidelity Estimation', 'Time used', 'Fidelity Estimation Time']
    to_print = ['fidelity', 'Base Resources Attempt', 'Fidelity Estimation Time']
    for key in to_print:
        print(f'{key}: {result[key]}')


In [44]:
visualize(results['0G'])

fidelity: 0.8196666666666667
Base Resources Attempt: 178955
Fidelity Estimation Time: 3.5787966666663524


In [31]:
results

{'0G': {'numPhysicalBuffer': 20,
  'numInternalEncodingBuffer': 20,
  'numInternalDetectingBuffer': 10,
  'numInternalInterfaceBuffer': 2,
  'memory_function': <function qwanta.qwanta.Xperiment.__init__.<locals>.memory_error_function(time, tau=inf)>,
  'gate_error': 0.1,
  'measurement_error': 0,
  'timeline': [{'Main Process': 'Generate physical Bell pair',
    'Edges': ['Node 0', 'Node 1'],
    'Num Trials': True,
    'Label in': 'Physical Qubit',
    'Label out': 'Physical',
    'Resource Type': nan,
    'Protocol': nan,
    'Note': nan,
    'Schedule Config': None,
    'isSuccess': 0},
   {'Main Process': 'Generate physical Bell pair',
    'Edges': ['Node 1', 'Node 2'],
    'Num Trials': True,
    'Label in': 'Physical Qubit',
    'Label out': 'Physical',
    'Resource Type': nan,
    'Protocol': nan,
    'Note': nan,
    'Schedule Config': None,
    'isSuccess': 0},
   {'Main Process': 'Entanglement swapping',
    'Edges': ['Node 0', 'Node 1', 'Node 2'],
    'Num Trials': True,
  

In [46]:
# print all generation results
for key in results.keys():
    print(f'Generation {key}')
    visualize(results[key])
    print('\n')

Generation 0G
fidelity: 0.8196666666666667
Base Resources Attempt: 178955
Fidelity Estimation Time: 3.5787966666663524


Generation 1G-Ss-Dp
fidelity: 0.6646666666666666
Base Resources Attempt: 693019
Fidelity Estimation Time: 13.95895000001088


Generation 2G-NonLocalCNOT
fidelity: 0.24466666666666667
Base Resources Attempt: 1264704
Fidelity Estimation Time: 29.07743333335874


Generation 1-2G-DirectedEncoded
fidelity: 0.24833333333333332
Base Resources Attempt: 690898
Fidelity Estimation Time: 13.889330000010748


Generation HG-END2ENDPurifiedEncoded
fidelity: 0.4666666666666667
Base Resources Attempt: 11154
Fidelity Estimation Time: 0.24849666666666453


Generation 1-2G-DirectedEncoded-Perfect
fidelity: 0.27599999999999997
Base Resources Attempt: 692104
Fidelity Estimation Time: 13.886480000010769


Generation 2G-NonLocalCNOT-Perfect
fidelity: 0.2495
Base Resources Attempt: 1264243
Fidelity Estimation Time: 29.173070000025593


